## G Colab

In [ ]:
print('Mounting Gdrive')
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/')


colab_env = True

## Lib

In [1]:
import os
import tensorflow as tf
import keras
import pickle

from mlflow import MlflowClient
import mlflow.tensorflow

2024-04-25 09:25:52.971077: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-25 09:25:53.004378: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 09:25:53.510819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.config.experimental.list_physical_devices('GPU')

2024-04-25 09:25:54.275700: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 09:25:54.312990: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 09:25:54.313137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Vars

In [4]:
epochs = 25
learning_rate = 1e-4

In [5]:
if colab_env == True:
    model_path = "/content/drive/MyDrive/ml_projects/mushroom-project/storage/models/builded/generated_model_efficientnetv2_21k_finetuned_1k_v1.keras"
    train_dataset_path = "/content/drive/MyDrive/ml_projects/mushroom-project/storage/datas/tf_datasets/train_dataset"
    val_dataset_path = "/content/drive/MyDrive/ml_projects/mushroom-project/storage/datas/tf_datasets/val_dataset"
    tracking_uri = "/content/drive/MyDrive/ml_projects/mushroom-project/storage/mlruns"
    model_checkpoint_url = "./content/drive/MyDrive/ml_projects/mushroom-project/storage/models/model_checkpoint"    

else:
    model_path = "../../storage/models/builded/generated_model_efficientnetv2_21k_finetuned_1k_v1.keras"
    train_dataset_path = "../../storage/datas/tf_datasets/train_dataset"
    val_dataset_path = "../../storage/datas/tf_datasets/val_dataset"
    tracking_uri = "../../storage/mlruns"
    model_checkpoint_url = "../../storage/models/model_checkpoint"

## Load model & Datasets

### Model

In [6]:
# Load Model
model = keras.saving.load_model(model_path)

2024-04-25 09:25:55.544497: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 4.00GiB (4294967296 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-04-25 09:25:55.544705: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 3.60GiB (3865470464 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-04-25 09:25:55.544865: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 3.24GiB (3478923264 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-04-25 09:25:55.545024: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 2.92GiB (3131030784 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-04-25 09:25:55.545188: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 2.62GiB (2817927680 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-04-25 09:25:55.545340: I 

In [7]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,889,754 (30.10 MB)

 Trainable params: 1,970,442 (7.52 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [8]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])

### Datasets

In [9]:
# Load datasets
train_datas = tf.data.Dataset.load(train_dataset_path)
val_datas = tf.data.Dataset.load(val_dataset_path)

## Fit

In [10]:
# Callbacks
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(model_checkpoint_url, "model_{epoch:02d}_{val_loss:.2f}.weights.h5"),
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=5)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=0)

In [11]:
# Start mlflow client
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient()
mlflow.tensorflow.autolog()

history = model.fit(
    train_datas,
    validation_data = val_datas,
    epochs = epochs,
    initial_epoch = 0,
    callbacks=[model_checkpoint, early_stopping, reduce_lr],
    verbose = 1)

Epoch 1/25


I0000 00:00:1714029967.677014   11863 service.cc:145] XLA service 0x7f0d74057430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714029967.677036   11863 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2024-04-25 09:26:07.941551: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-25 09:26:08.765587: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


: 

In [ ]:
# Fine tuning
fine_tuned_model = model

for layer in fine_tuned_model.layers:
    layer.trainable = False
    
fine_tuned_model.layers[1].trainable = True

fine_tuned_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])

In [ ]:
history_fine_tune = model.fit(
    train_datas,
    validation_data = val_datas,
    epochs = epochs,
    initial_epoch = 0,
    callbacks=[model_checkpoint, early_stopping, reduce_lr],
    verbose = 1)

### Save Model

In [ ]:
# Save model
trained_model_path = "../../storage/models/trained/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained.keras"
fine_tuned_model_path = "../../storage/models/trained/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained_fine_tuned.keras"
model.save(filepath = trained_model_path, overwrite = True)
fine_tuned_model.save(filepath = fine_tuned_model_path, overwrite = True)

# Save history
history_path = "../../storage/models/histories/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained_history.pickle"
with open(history_path, 'wb') as file:
    pickle.dump(history.history, file)

history_fine_tune_path = "../../storage/models/histories/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained_fine_tuned_history.pickle"
with open(history_fine_tune_path, 'wb') as file:
    pickle.dump(history_fine_tune.history, file)